# **Image Optimization & DB 2019**
## **1 Gallery 2019 DataBases**
```
GET Text Data
```

In [1]:
import requests, re
from lxml.html import fromstring, tostring

# Link 및 Text 정보 수집하기
def get_response(url:str, xpath:str, get=None):
    result = []
    response      = requests.get(url).content.decode('utf-8')
    response_lxml = fromstring(response) # tostring(_, encoding='unicode')
    response_html = response_lxml.xpath(xpath)
    if get:
        for _html in response_html:
            result.append(_html.get(get))
        return result
    else:
        return response_html

In [2]:
import re, os

# 파일 다운로드 및 이름 바꾸기
def change_file_name(link, rename=None):
    r"""Image Links Change to 'rename' file"""
    # Download the Link file
    os.system(f"wget {link}")

    # Get the File name components
    tokenizer = re.compile("[A-z0-9]+\.(?:jpg|gif|png|jpeg)")
    file_full_name = "".join(tokenizer.findall(link))
    file_name = file_full_name.split('.')[0]
    file_format = file_full_name.split('.')[-1]
    
    # Changing the File name
    file_full_name = file_full_name.replace('.', '\.') # 원본 regex 문법에 맞게 변경
    if rename:
        rename = rename + '\.' + file_format   # 대상 확장자 추가 및 regex 문법에 맞게 변경
        os.system(f"mv {file_full_name} {rename}")
    else:
        pass

In [3]:
url = "http://arterior-ga.com/97"
gallery_xpath = '//div[@doz_type="widget"]//div[contains(@class, "grid_02") and contains(@class, "img_border")]'
gallery_photo_xpath = './/div[contains(@class, "item_container") and contains(@class, "_item_container")]'
gallery_photo_xpath ='.//div[contains(@class,"_item") and contains(@class, "item_gallary")]'
# Gallery 영역 찾기 > Gallery 내부의 Photo 영역 찾기
response = get_response(url, gallery_xpath)[0] # tostring(response[0], encoding='unicode')  # 내용의 확인
response = response.xpath(gallery_photo_xpath)
len(response)

37

## **2 Gallery Data From Each Photos**
개별 사진에서 Photo 사진 및 Text 추출하기

In [4]:
regex_imweb = "https?:\/\/(?:www\.|(?!www))cdn\.imweb.me\/thumbnail\/[a-zA-Z0-9-]+\/[a-zA-Z0-9-]+\.(?:jpg|gif|png|jpeg)"
result = []
for photo in response:
    # photo_url
    photo_url = photo.xpath('.//div[contains(@class, "_img_wrap")]')[0].get('style')
    photo_url = "".join(re.findall(regex_imweb, photo_url))
    # photo_title
    photo_title = tostring(photo.xpath('.//h4')[0], encoding='unicode')
    # photo_description
    photo_title = tostring(photo.xpath('.//p[@class="title"]')[0], encoding='unicode')
    result.append([photo_url, photo_title])
result[::10]

[['https://cdn.imweb.me/thumbnail/20200327/4166b65ca0f48.jpg',
  '<p class="title">내부미팅<span class="body">첫 내부미팅입니다. 약간 교주님(?) 같으신 꽃별대표님!</span></p>'],
 ['https://cdn.imweb.me/thumbnail/20200327/90e3e846a9cb0.jpg',
  '<p class="title">워크숍</p>'],
 ['https://cdn.imweb.me/thumbnail/20200327/20250c78573b2.jpg',
  '<p class="title">개별미팅<span class="body">#박호은작가</span></p>'],
 ['https://cdn.imweb.me/thumbnail/20200327/3ab8519013fe3.jpg',
  '<p class="title">현장설치<span class="body">동네사람들 모두 구경도 나오고요. </span></p>']]

## **3 Saving Data Set**
추출한 자료를 바탕으로 저장하기

In [5]:
import os
from tqdm import tqdm
links = [_[0]   for _ in result]

for no, _ in tqdm(enumerate(links)):
    change_file_name(_, f'gallery{no:02d}')

37it [00:02, 14.40it/s]


In [17]:
import pandas as pd
table = pd.DataFrame(result)

In [18]:
regex_html = re.compile(r'<.*?>')
table[1] = [regex_html.sub('', _)  for _ in table[1]]
table.head()

,0,1
0,https://cdn.imweb.me/thumbnail/20200327/4166b6...,내부미팅첫 내부미팅입니다. 약간 교주님(?) 같으신 꽃별대표님!
1,https://cdn.imweb.me/thumbnail/20200327/0125c8...,면접일
2,https://cdn.imweb.me/thumbnail/20200327/800b64...,면접일
3,https://cdn.imweb.me/thumbnail/20200327/548416...,설명회관악구청에서 소상공인 대상 설명회를 진행했습니다.
4,https://cdn.imweb.me/thumbnail/20200327/116d28...,오리엔테이션


In [19]:
table.to_csv('gallery.csv')